In [4]:
import torch
import torchvision
from vggNet import VGG

from torch.utils.data import DataLoader, random_split
# from torchvision import datasets
from torchvision import transforms

from tqdm import tqdm

In [5]:
vgg = VGG('C', add_norm = True, n_classes = 120)
vgg

VGG(
  (convnet): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 

In [4]:
trans = transforms.Compose([transforms.Resize((224, 224)),
                            transforms.ToTensor(),
                            transforms.Normalize((0.485,0.456,0.406), (0.229, 0.224, 0.225))])
image_data = torchvision.datasets.ImageFolder(root = './data/Images', transform = trans)

In [5]:
train_size = int(image_data.__len__() * 0.8)
test_size = image_data.__len__() - train_size
train_data, test_data = random_split(image_data, [train_size, test_size])

In [6]:
train_dataloader = DataLoader(train_data, batch_size = 2, shuffle = True)
test_dataloader = DataLoader(test_data, batch_size = 2, shuffle = True)

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [8]:
vgg = vgg.to(device)

epochs = 1
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(vgg.parameters(), lr = 1e-3)

In [ ]:
vgg.train()
for epoch in tqdm(range(epochs)):
    running_loss = 0.0
    
    for i, data in enumerate(train_dataloader):
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = vgg(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        if loss.item() > 1000:
            print(loss.item())
            for param in vgg.parameters():
                print(param.data)
        
        running_loss += loss.item()
        if i % 100 == 0:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
print('Training Finishied')

In [11]:
correct = 0
total = 0
# 학습 중이 아니므로, 출력에 대한 변화도를 계산할 필요가 없습니다
with torch.no_grad():
    for data in tqdm(test_dataloader):
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        # 신경망에 이미지를 통과시켜 출력을 계산합니다
        outputs = vgg(images)
        # 가장 높은 값(energy)를 갖는 분류(class)를 정답으로 선택하겠습니다
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct // total} %')

100%|██████████| 2058/2058 [01:18<00:00, 26.09it/s]

Accuracy of the network on the test images: 1 %


In [13]:
classes = image_data.classes

# 각 분류(class)에 대한 예측값 계산을 위해 준비
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# 변화도는 여전히 필요하지 않습니다
with torch.no_grad():
    for data in tqdm(test_dataloader):
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        
        outputs = vgg(images)
        _, predictions = torch.max(outputs, 1)
        # 각 분류별로 올바른 예측 수를 모읍니다
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

100%|██████████| 2058/2058 [01:15<00:00, 27.24it/s]

Accuracy for class: n02085620-Chihuahua is 0.0 %
Accuracy for class: n02085782-Japanese_spaniel is 0.0 %
Accuracy for class: n02085936-Maltese_dog is 0.0 %
Accuracy for class: n02086079-Pekinese is 0.0 %
Accuracy for class: n02086240-Shih-Tzu is 0.0 %
Accuracy for class: n02086646-Blenheim_spaniel is 0.0 %
Accuracy for class: n02086910-papillon is 0.0 %
Accuracy for class: n02087046-toy_terrier is 100.0 %
Accuracy for class: n02087394-Rhodesian_ridgeback is 0.0 %
Accuracy for class: n02088094-Afghan_hound is 0.0 %
Accuracy for class: n02088238-basset is 0.0 %
Accuracy for class: n02088364-beagle is 0.0 %
Accuracy for class: n02088466-bloodhound is 0.0 %
Accuracy for class: n02088632-bluetick is 0.0 %
Accuracy for class: n02089078-black-and-tan_coonhound is 0.0 %
Accuracy for class: n02089867-Walker_hound is 0.0 %
Accuracy for class: n02089973-English_foxhound is 0.0 %
Accuracy for class: n02090379-redbone is 0.0 %
Accuracy for class: n02090622-borzoi is 0.0 %
Accuracy for class: n02090

In [14]:
# 각 분류별 정확도(accuracy)를 출력합니다
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: n02085620-Chihuahua is 0.0 %
Accuracy for class: n02085782-Japanese_spaniel is 0.0 %
Accuracy for class: n02085936-Maltese_dog is 0.0 %
Accuracy for class: n02086079-Pekinese is 0.0 %
Accuracy for class: n02086240-Shih-Tzu is 0.0 %
Accuracy for class: n02086646-Blenheim_spaniel is 0.0 %
Accuracy for class: n02086910-papillon is 0.0 %
Accuracy for class: n02087046-toy_terrier is 100.0 %
Accuracy for class: n02087394-Rhodesian_ridgeback is 0.0 %
Accuracy for class: n02088094-Afghan_hound is 0.0 %
Accuracy for class: n02088238-basset is 0.0 %
Accuracy for class: n02088364-beagle is 0.0 %
Accuracy for class: n02088466-bloodhound is 0.0 %
Accuracy for class: n02088632-bluetick is 0.0 %
Accuracy for class: n02089078-black-and-tan_coonhound is 0.0 %
Accuracy for class: n02089867-Walker_hound is 0.0 %
Accuracy for class: n02089973-English_foxhound is 0.0 %
Accuracy for class: n02090379-redbone is 0.0 %
Accuracy for class: n02090622-borzoi is 0.0 %
Accuracy for class: n02090